<a href="https://colab.research.google.com/github/trailblazerakash/MachineLearning/blob/master/hdfc/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
%matplotlib inline

from itertools import chain
from sklearn import preprocessing, ensemble
from sklearn.preprocessing import OneHotEncoder




In [3]:

train_df = pd.read_csv('/content/drive/My Drive/HDFC/d407ebe0db9a11e9/DataSet/Train.csv')
train_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,...,Col2358,Col2359,Col2360,Col2361,Col2362,Col2363,Col2364,Col2365,Col2366,Col2367,Col2368,Col2369,Col2370,Col2371,Col2372,Col2373,Col2374,Col2375,Col2376,Col2377,Col2378,Col2379,Col2380,Col2381,Col2382,Col2383,Col2384,Col2385,Col2386,Col2387,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,RIGHGSN,1,1.086711e+04,8.648345e+03,1.576618e+04,2.890466e+04,5.273655e+03,2.346153e+04,1.285597e+04,2.871509e+04,9.114586e+03,3.043078e+04,9.293496,8.570479,10.063117,10.271758,0.548538,0.403366,NaN,NaN,1.486524,1.728324,0.993441,1.297050,0.137398,0.115987,0.260428,1.414214,1,1,1,1,1,1,0,6,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,214.0,214.0,68.0,22.0,22.0,7.0
1,RIMHI5ZGP,0,2.584312e+05,2.179633e+05,1.866287e+05,1.866287e+05,2.154519e+05,1.923615e+05,1.726538e+05,1.666974e+05,2.013518e+05,1.859404e+05,12.462385,12.280493,12.167132,12.136876,1.167898,2.676398,NaN,NaN,0.792123,0.934556,0.893203,0.966620,0.753892,0.808295,1.414214,NaN,1,1,3,4,4,0,4,5,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.0,41.0,38.0,12.0,12.0,11.0
2,RIHOG1SZU,0,1.459355e+07,1.260060e+07,1.100315e+07,1.010315e+07,1.879882e+07,1.514035e+07,5.320778e+06,6.906324e+06,9.368338e+06,1.113330e+07,16.496090,16.749305,16.532874,16.128357,1.145182,2.203874,NaN,NaN,0.422264,0.498347,0.683582,0.735339,0.808115,0.776910,1.414214,NaN,3,3,4,12,4,0,2,4,NaN,1.0,NaN,NaN,...,1,0,1,0,1,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,1,0,378.0,378.0,310.0,21.0,21.0,16.0
3,RIHOLMQWU,0,6.727043e+05,8.181116e+05,6.794893e+05,6.794893e+05,8.257254e+05,7.605803e+05,3.577134e+05,4.236612e+05,4.304848e+05,4.065539e+05,13.419061,13.624018,13.541837,13.429097,1.204010,0.536304,NaN,NaN,0.437243,0.521341,0.623499,0.534531,-0.125829,0.388830,-1.414214,NaN,2,2,2,24,1,0,1,4,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1,2,0,1,2,0,0,0,0,0,1,2,0,0,0,0,0,1,2,0,0,0,0,0,0,702.0,702.0,644.0,951.0,951.0,743.0
4,RIHO584ET,0,-1.218742e+06,-3.005361e+06,-1.666241e+05,-2.403574e+06,2.583183e+06,2.086546e+06,5.938358e+06,5.446532e+06,3.473444e+06,3.262168e+06,NaN,14.764533,14.551021,NaN,NaN,NaN,NaN,NaN,NaN,1.344637,NaN,1.563430,-0.113101,0.617486,1.414214,NaN,4,3,4,68,8,0,2,4,NaN,3.0,NaN,NaN,...,0,0,0,0,0,0,2,1,0,0,0,0,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1305.0,1305.0,1211.0,2205.0,2205.0,1831.0


In [0]:
df_dtypes = pd.DataFrame(train_df.dtypes,columns=['dtypes'])
df_dtypes['first_value'] = train_df.loc[0].values


In [0]:
df_dtypes['Null_Count'] = train_df[train_df.columns].isnull().sum()
   

In [6]:
df_dtypes.head()

,dtypes,first_value,Null_Count
Col1,object,RIGHGSN,0
Col2,int64,1,0
Col3,float64,10867.1,0
Col4,float64,8648.34,0
Col5,float64,15766.2,0


In [7]:
print (df.isna())

NameError: ignored

In [0]:
df.isna().mean()

In [0]:
tdf = train_df.loc[:, train_df.isna().mean() < .25]

In [10]:
tdf.shape

(17521, 2115)

In [0]:
#NOT WORKING PROPERLY

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]


In [0]:

print("Top Absolute Correlations")
print(get_top_abs_correlations(tdf, 100))

In [11]:
dtype_df = tdf.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()

,Column Type,Count
0,int64,1548
1,float64,566
2,object,1


In [12]:
dtype_df = tdf.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()


,Column Type,Count
0,int64,1548
1,float64,566
2,object,1


In [13]:
dtype_df.ix[:10,:]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,Count,Column Type
0,Col1,object
1,Col2,int64
2,Col3,float64
3,Col4,float64
4,Col5,float64
5,Col6,float64
6,Col7,float64
7,Col8,float64
8,Col9,float64
9,Col10,float64


In [14]:
unique_values_dict = {}
for col in tdf.columns:
    if col not in ["Col1","Col2"]:
        unique_value = str(np.sort(tdf[col].unique()).tolist())
        tlist = unique_values_dict.get(unique_value, [])
        tlist.append(col)
        unique_values_dict[unique_value] = tlist[:]
for unique_val, columns in unique_values_dict.items():
    print("Columns containing the unique values : ",unique_val)
    print(columns)
    print("--------------------------------------------------")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
correlation_threshold = 0.99 #
train_integer = tdf.drop(["Col1","Col2"],axis=1)
cor = train_integer.corr()

In [17]:

cor.loc[:,:] =  np.tril(cor, k=-1)
cor = cor.stack()
correlated = cor[cor > correlation_threshold].reset_index().loc[:,['level_0','level_1']]
correlated = correlated.query('level_0 not in level_1')
correlated_array =  correlated.groupby('level_0').agg(lambda x: set(chain(x.level_0,x.level_1))).values
correlated_array

array([[{'Col946', 'Col1038'}],
       [{'Col950', 'Col1042'}],
       [{'Col982', 'Col1074'}],
       [{'Col1075', 'Col983'}],
       [{'Col108', 'Col94'}],
       [{'Col1080', 'Col988'}],
       [{'Col1081', 'Col989'}],
       [{'Col1088', 'Col996'}],
       [{'Col997', 'Col1089'}],
       [{'Col998', 'Col1090'}],
       [{'Col984', 'Col1168', 'Col1076'}],
       [{'Col993', 'Col1177', 'Col1085'}],
       [{'Col1086', 'Col994', 'Col1178'}],
       [{'Col1188', 'Col1096'}],
       [{'Col1084', 'Col1268', 'Col1176', 'Col992'}],
       [{'Col1288', 'Col1104', 'Col1196'}],
       [{'Col129', 'Col121'}],
       [{'Col1292', 'Col1200'}],
       [{'Col131', 'Col123'}],
       [{'Col1006', 'Col1326', 'Col1310'}],
       [{'Col125', 'Col133'}],
       [{'Col1022', 'Col1334'}],
       [{'Col942', 'Col1344'}],
       [{'Col1368', 'Col962', 'Col1352'}],
       [{'Col137', 'Col136'}],
       [{'Col970', 'Col1370'}],
       [{'Col1376', 'Col978'}],
       [{'Col1302', 'Col1078', 'Col986', 'Col1392

In [18]:
correlated_features = []
for sets in correlated_array:
    element_list = list(sets[0])
    for idx, el in enumerate(element_list):
        if idx is not 0:
            correlated_features.append(el)

print (correlated_features)


['Col1038', 'Col1042', 'Col1074', 'Col983', 'Col94', 'Col988', 'Col989', 'Col996', 'Col1089', 'Col1090', 'Col1168', 'Col1076', 'Col1177', 'Col1085', 'Col994', 'Col1178', 'Col1096', 'Col1268', 'Col1176', 'Col992', 'Col1104', 'Col1196', 'Col121', 'Col1200', 'Col123', 'Col1326', 'Col1310', 'Col133', 'Col1334', 'Col1344', 'Col962', 'Col1352', 'Col136', 'Col1370', 'Col978', 'Col1078', 'Col986', 'Col1392', 'Col1394', 'Col1416', 'Col1098', 'Col1328', 'Col1106', 'Col1424', 'Col1428', 'Col1432', 'Col1034', 'Col1442', 'Col1458', 'Col1460', 'Col1070', 'Col1482', 'Col1396', 'Col1486', 'Col1308', 'Col1488', 'Col1504', 'Col1506', 'Col1490', 'Col1508', 'Col1206', 'Col1430', 'Col1126', 'Col1146', 'Col1532', 'Col1154', 'Col1162', 'Col1262', 'Col1492', 'Col1406', 'Col1496', 'Col990', 'Col1174', 'Col1590', 'Col1500', 'Col1266', 'Col1320', 'Col1082', 'Col1322', 'Col1502', 'Col1412', 'Col1282', 'Col1596', 'Col1598', 'Col1510', 'Col1330', 'Col1600', 'Col1511', 'Col1331', 'Col1421', 'Col1604', 'Col161', 'Col

In [0]:
non_cor_train_df = tdf.drop(correlated_features, axis=1 )

In [20]:
non_cor_train_df.shape


(17521, 1369)

In [48]:
non_cor_train_df.head()

,Col1,Col2,Col3,Col4,Col5,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col42,Col43,Col44,Col45,Col46,Col47,Col48,Col49,Col50,Col51,...,Col2318,Col2321,Col2323,Col2324,Col2327,Col2329,Col2336,Col2337,Col2338,Col2340,Col2341,Col2342,Col2346,Col2347,Col2348,Col2350,Col2351,Col2354,Col2356,Col2358,Col2360,Col2361,Col2362,Col2363,Col2364,Col2366,Col2368,Col2370,Col2376,Col2377,Col2378,Col2380,Col2382,Col2383,Col2384,Col2387,Col2392,Col2394,Col2395,Col2397
0,RIGHGSN,1,1.086711e+04,8.648345e+03,1.576618e+04,1.285597e+04,2.871509e+04,9.114586e+03,3.043078e+04,9.293496,8.570479,10.063117,10.271758,0.548538,0.403366,1.486524,1.728324,0.993441,1.297050,0.137398,0.115987,0.260428,1,1,1,1,1,1,0,6,6.07,6.07,2,3,2,3,2,2,2,2,...,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.0,68.0,22.0,7.0
1,RIMHI5ZGP,0,2.584312e+05,2.179633e+05,1.866287e+05,1.726538e+05,1.666974e+05,2.013518e+05,1.859404e+05,12.462385,12.280493,12.167132,12.136876,1.167898,2.676398,0.792123,0.934556,0.893203,0.966620,0.753892,0.808295,1.414214,1,1,3,4,4,0,4,5,116.65,116.65,3,0,2,0,2,0,1,0,...,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.0,38.0,12.0,11.0
2,RIHOG1SZU,0,1.459355e+07,1.260060e+07,1.100315e+07,5.320778e+06,6.906324e+06,9.368338e+06,1.113330e+07,16.496090,16.749305,16.532874,16.128357,1.145182,2.203874,0.422264,0.498347,0.683582,0.735339,0.808115,0.776910,1.414214,3,3,4,12,4,0,2,4,33.28,33.28,2,1,2,0,1,0,1,0,...,3,0,0,2,0,1,2,0,1,0,0,0,1,0,2,0,0,0,0,1,1,0,1,0,0,0,2,1,0,0,2,0,0,0,0,0,378.0,310.0,21.0,16.0
3,RIHOLMQWU,0,6.727043e+05,8.181116e+05,6.794893e+05,3.577134e+05,4.236612e+05,4.304848e+05,4.065539e+05,13.419061,13.624018,13.541837,13.429097,1.204010,0.536304,0.437243,0.521341,0.623499,0.534531,-0.125829,0.388830,-1.414214,2,2,2,24,1,0,1,4,-7.75,-7.75,2,1,2,1,1,1,1,1,...,2,0,0,1,0,2,3,0,1,0,0,1,0,1,0,1,2,0,0,0,0,0,0,0,0,0,2,1,0,1,2,0,0,0,1,0,702.0,644.0,951.0,743.0
4,RIHO584ET,0,-1.218742e+06,-3.005361e+06,-1.666241e+05,5.938358e+06,5.446532e+06,3.473444e+06,3.262168e+06,NaN,14.764533,14.551021,NaN,NaN,NaN,NaN,1.344637,NaN,1.563430,-0.113101,0.617486,1.414214,4,3,4,68,8,0,2,4,NaN,NaN,1,2,1,2,1,2,1,2,...,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,1305.0,1211.0,2205.0,1831.0


In [0]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)
g = sns.pairplot(non_cor_train_df)

In [22]:
unique_values_list = non_cor_train_df.nunique()
unique_values_list=unique_values_list.reset_index()
unique_values_list.columns = ['Col',"Count"]
unique_values_list['Col'].count()

1369

In [0]:
continious =  unique_values_list[unique_values_list['Count'] > 990]
cont_cols = continious['Col']
cont_df = non_cor_train_df[cont_cols]
cat_df = non_cor_train_df.drop(cont_cols,axis=1)

In [24]:
wrongly_classified =['Col2394','Col2392' ,'Col2395','Col2397']
for col in wrongly_classified:
  cont_df[col] = non_cor_train_df[col]
  cat_df.drop(col,axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
cat_df = cat_df.drop('Col2', axis=1)

In [0]:
def get_missing_count(df):
  missing_df = df.isnull().sum(axis=0).reset_index()
  missing_df.columns = ['column_name', 'missing_count']
  missing_df = missing_df.loc[missing_df['missing_count']>0]
  return missing_df

1367

In [0]:
def get_non_missing_columns(df):
  missing_df = df.isnull().sum(axis=0).reset_index()
  missing_df.columns = ['column_name', 'missing_count']
  missing_df = missing_df.loc[missing_df['missing_count']==0]
  return missing_df

In [0]:
cat_non_miss_columns =list(get_non_missing_columns(cat_df)['column_name'])
cont_non_miss_columns =list(get_non_missing_columns(cont_df)['column_name'])



In [49]:
final_cols = cat_non_miss_columns+cont_non_miss_columns

training_df = non_cor_train_df[final_cols]
training_df['Col2'] = non_cor_train_df['Col2']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [50]:
training_df.head()

,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col44,Col45,Col46,Col47,Col48,Col49,Col50,Col51,Col52,Col53,Col54,Col55,Col56,Col57,Col58,Col59,Col60,Col61,Col62,Col63,Col64,Col65,Col66,Col67,Col68,Col69,Col70,Col71,Col72,Col73,Col74,Col75,...,Col2364,Col2366,Col2368,Col2370,Col2376,Col2377,Col2378,Col2380,Col2382,Col2383,Col2384,Col2387,Col1,Col3,Col4,Col5,Col9,Col10,Col11,Col12,Col25,Col26,Col169,Col181,Col196,Col198,Col200,Col207,Col397,Col399,Col402,Col575,Col589,Col590,Col591,Col596,Col598,Col599,Col600,Col2
0,1,1,1,1,1,1,0,6,2,3,2,3,2,2,2,2,1,3,1,3,1,2,1,2,1,1,1,1,1,1,1,1,3,1,3,1,3,1,3,1,...,0,0,0,0,0,0,0,0,0,0,0,0,RIGHGSN,1.086711e+04,8.648345e+03,1.576618e+04,1.285597e+04,2.871509e+04,9.114586e+03,3.043078e+04,0.137398,0.115987,6.247142e+05,1.024016e+05,4166.666667,85475.99833,0.0,8.575733e+04,0.0,0.00,12166.66667,6.460633e+04,5000.0,0.000000e+00,59606.33333,0.000000e+00,5000.00000,5.960633e+04,5.960633e+04,1
1,1,1,3,4,4,0,4,5,3,0,2,0,2,0,1,0,3,0,2,0,2,0,1,0,2,0,2,0,2,0,1,0,2,0,2,0,2,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,RIMHI5ZGP,2.584312e+05,2.179633e+05,1.866287e+05,1.726538e+05,1.666974e+05,2.013518e+05,1.859404e+05,0.753892,0.808295,5.282476e+06,9.443964e+05,0.000000,0.00000,0.0,6.251958e+05,0.0,23.00,0.00000,1.542111e+06,0.0,1.862743e+05,0.00000,5.319627e+05,0.00000,1.862743e+05,1.010149e+06,0
2,3,3,4,12,4,0,2,4,2,1,2,0,1,0,1,0,2,1,2,0,1,0,1,0,1,1,1,0,1,0,1,0,2,1,2,1,1,0,1,0,...,0,0,2,1,0,0,2,0,0,0,0,0,RIHOG1SZU,1.459355e+07,1.260060e+07,1.100315e+07,5.320778e+06,6.906324e+06,9.368338e+06,1.113330e+07,0.808115,0.776910,2.226136e+08,3.836221e+07,0.000000,25704.00000,2000000.0,1.896200e+07,2000000.0,1116.08,31000.00000,4.933168e+07,0.0,6.631934e+06,42840.00000,1.064490e+05,41666.66667,6.674774e+06,2.315162e+07,0
3,2,2,2,24,1,0,1,4,2,1,2,1,1,1,1,1,2,1,2,1,1,1,1,1,2,0,2,0,1,0,1,0,2,1,2,1,2,1,2,1,...,0,0,2,1,0,1,2,0,0,0,1,0,RIHOLMQWU,6.727043e+05,8.181116e+05,6.794893e+05,3.577134e+05,4.236612e+05,4.304848e+05,4.065539e+05,-0.125829,0.388830,2.821560e+07,5.511437e+06,0.000000,25582.30400,0.0,5.178721e+06,0.0,891.00,0.00000,7.301997e+06,0.0,5.771387e+05,42637.17333,4.420246e+05,0.00000,6.197758e+05,6.803684e+06,0
4,4,3,4,68,8,0,2,4,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,0,2,0,2,0,2,0,2,1,2,1,2,1,2,1,1,...,2,0,0,2,0,0,0,0,0,0,0,0,RIHO584ET,-1.218742e+06,-3.005361e+06,-1.666241e+05,5.938358e+06,5.446532e+06,3.473444e+06,3.262168e+06,-0.113101,0.617486,4.726991e+08,9.346137e+07,0.000000,0.00000,0.0,5.305945e+07,0.0,3079.85,121000.00000,1.141207e+08,0.0,0.000000e+00,0.00000,5.734043e+07,201666.66670,0.000000e+00,5.339928e+07,0


In [0]:
enc=OneHotEncoder(sparse=False)

In [0]:
for col in cat_non_miss_columns:

       data=X_train[[col]].append(X_test[[col]])
       enc.fit(data)

       temp = enc.transform(X_train[[col]])

       temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in data[col]
            .value_counts().index])


       temp=temp.set_index(X_train.index.values)
       
       X_train_1=pd.concat([X_train_1,temp],axis=1)

       temp = enc.transform(X_test[[col]])
       
       temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in data[col]
            .value_counts().index])

       temp=temp.set_index(X_test.index.values)

       X_test_1=pd.concat([X_test_1,temp],axis=1)

column_name      65
missing_count    65
dtype: int64

In [0]:

get_missing_count(cont_df).count()

column_name      290
missing_count    290
dtype: int64